# Routing Topology: OSPF using FRRouting

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


## Import the FABlib Library


In [11]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/work/fabric_config/tokens.json
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
Bastion Username,pruth_0031379841
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Create the Experiment Slice

The following creates private layer 2 networks on three sites including OSPF gateway routers that propagate routes across the topology. 


In [12]:
slice_name = 'OSPF_Routing_Topology1'

sites= fablib.get_random_sites(count=3, avoid=[])
print(f"Sites: {sites}")

sites=['DALL','WASH','UTAH']

router_base_name='router'
router_link_base_name='link'

node_base_name='node'
local_network_base_name='net_local'

site_node_count=1


rocky_repo_ip=IPv4Address('10.133.130.2')



Sites: ['TACC', 'GPN', 'DALL']


In [13]:

slice = fablib.new_slice(name=slice_name)

# Create Routers
routers = []
frr_user_data = {}
for i, site in enumerate(sites):
    router_name = f"{router_base_name}{site}"
    frr_user_data[router_name] = {}
    local_subnet = IPv4Network(f"192.168.{i+1}.0/24")
    local_gateway = local_subnet[1]
    
    # Add control net
    control_net = slice.add_l3network(name=f'control_net_{site}', type='IPv4')
    router = slice.add_node(name=router_name, site=site, cores=16, ram=32, disk=100).enable_docker()
    iface_control = router.add_component(model='NIC_Basic', name='nic_control').get_interfaces()[0]
    control_net.add_interface(iface_control)
    iface_control.set_mode('auto')
    router.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=control_net.get_gateway())
    if rocky_repo_ip:
        router.set_rocky_repo(rocky_repo_ip)


    # Add experiment nets
    iface_local = router.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]
    iface_local.set_mode('fablib')
    
    local_net = slice.add_l2network(name=f'{local_network_base_name}{site}', subnet=local_subnet, gateway=local_gateway)
    local_net.add_interface(iface_local)
    iface_local.set_ip_addr(local_gateway)
    
    frr_user_data[router_name]['local_iface'] = iface_local.get_name()
    frr_user_data[router_name]['link_ifaces'] = []
    
    routers.append(router)

# Create Links between routers (ring)
links = []
for i, site in enumerate(sites):
    link_info = {}
    link_subnet = IPv4Network(f"192.168.10{i+1}.0/24")
    
    router1 = routers[i]
    router2 = routers[(i+1)%len(sites)]
    
    #link_info['name'] = f'{router_link_base_name}{i+1}'
    link_info['name'] = f'{router1.get_site()}-{router2.get_site()}-{router_link_base_name}{i+1}'

    
    link = slice.add_l2network(name=link_info['name'], subnet=link_subnet)
    links.append(link)

    iface1 = router1.add_component(model='NIC_Basic', name=link_info['name']).get_interfaces()[0]
    iface2 = router2.add_component(model='NIC_Basic', name=link_info['name']).get_interfaces()[0]
    
    iface1.set_mode('fablib')
    iface2.set_mode('fablib')
    
    link.add_interface(iface1)
    link.add_interface(iface2)
    
    iface1.set_ip_addr(link_subnet[1])
    iface2.set_ip_addr(link_subnet[2])
        
    frr_user_data[router1.get_name()]['link_ifaces'].append(iface1.get_name()) 
    frr_user_data[router2.get_name()]['link_ifaces'].append(iface2.get_name()) 

# Set frr user_data
for router in slice.get_nodes():
    user_data = router.get_user_data()
    user_data['frr'] = frr_user_data[router.get_name()]
    router.set_user_data(user_data)
    
#slice_id = slice.submit()


In [14]:
for i, site in enumerate(sites):
    for node_num in range(site_node_count):
        node_name = f"{site.lower()}{node_num+1}"
        node = slice.add_node(name=node_name, site=site, cores=16, ram=32, disk=100).enable_docker()
        
        iface = node.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
        network = slice.get_network(name=f'{local_network_base_name}{site}')
        network.add_interface(iface)
        iface.set_mode('auto')

        node.add_route(subnet=IPv4Network('192.168.0.0/16'), next_hop=network.get_gateway())
        
        # Add control net
        control_net = slice.get_network(f'control_net_{site}')
        iface_control = node.add_component(model='NIC_Basic', name='nic_control').get_interfaces()[0]
        control_net.add_interface(iface_control)
        iface_control.set_mode('auto')
        node.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=control_net.get_gateway())
        if rocky_repo_ip:
            node.set_rocky_repo(rocky_repo_ip)

        
slice.submit()


Retry: 5, Time: 1011 sec


ID,d55472a4-e574-4391-9ea1-0c9427ea8686
Name,OSPF_Routing_Topology1
Lease Expiration (UTC),2023-03-03 17:47:34 +0000
Lease Start (UTC),2023-03-02 17:47:36 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
bbcafbde-11d2-4e32-bbc6-62245cf3a2dd,routerDALL,16,32,100,default_rocky_8,qcow2,dall-w2.fabric-testbed.net,DALL,rocky,2001:400:a100:3000:f816:3eff:feee:6e99,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3000:f816:3eff:feee:6e99,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a51d165d-adae-46b8-af02-f11897089ce9,routerWASH,16,32,100,default_rocky_8,qcow2,wash-w2.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe08:e227,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe08:e227,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c0da2c1c-e0e5-482f-8931-affe216645e8,routerUTAH,16,32,100,default_rocky_8,qcow2,utah-w3.fabric-testbed.net,UTAH,rocky,2001:1948:417:7:f816:3eff:fea4:39dc,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:1948:417:7:f816:3eff:fea4:39dc,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
4af7503e-33e7-438b-87d1-a4b0d43fb8de,dall1,16,32,100,default_rocky_8,qcow2,dall-w2.fabric-testbed.net,DALL,rocky,2001:400:a100:3000:f816:3eff:fe8f:f75d,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3000:f816:3eff:fe8f:f75d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
318ec4d3-c2b0-4b2c-b542-9dae262b5e8b,wash1,16,32,100,default_rocky_8,qcow2,wash-w2.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe62:5bfa,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe62:5bfa,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
38715eaa-e085-4806-89ee-c83f974a84ba,utah1,16,32,100,default_rocky_8,qcow2,utah-w3.fabric-testbed.net,UTAH,rocky,2001:1948:417:7:f816:3eff:fe84:baa1,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:1948:417:7:f816:3eff:fe84:baa1,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
52423b3d-16be-4b16-b30c-a231e2d38b59,control_net_DALL,L3,FABNetv4,DALL,10.133.132.0/24,10.133.132.1,Active,
37cdbe51-88bd-4d86-8b0d-45a17232ebff,net_localDALL,L2,L2Bridge,DALL,192.168.1.0/24,192.168.1.1,Active,
b6a845e0-4bc0-44d4-b555-ba022893ac1c,control_net_WASH,L3,FABNetv4,WASH,10.133.2.0/24,10.133.2.1,Active,
c26ba511-d168-4acb-b975-d35cbd9d555c,net_localWASH,L2,L2Bridge,WASH,192.168.2.0/24,192.168.2.1,Active,
cd1f6c8e-d8eb-43c0-a48b-dc2ab89cbf04,control_net_UTAH,L3,FABNetv4,UTAH,10.132.1.0/24,10.132.1.1,Active,
d39d8d2d-8b3f-4aa7-93cf-97d47c42904d,net_localUTAH,L2,L2Bridge,UTAH,192.168.3.0/24,192.168.3.1,Active,
0f5e609a-d168-44bc-aec7-92c2dabc92db,DALL-WASH-link1,L2,L2STS,None,192.168.101.0/24,None,Active,
e0d73e01-34b4-4051-9dd2-eab0974fcf56,WASH-UTAH-link2,L2,L2STS,None,192.168.102.0/24,None,Active,
106618b2-7b85-4123-9c15-63102b6ac921,UTAH-DALL-link3,L2,L2STS,None,192.168.103.0/24,None,Active,


Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address
routerDALL-UTAH-DALL-link3-p1,routerDALL,UTAH-DALL-link3,100,fablib,,12:9C:F6:96:97:57,eth2,eth2,192.168.103.2
routerDALL-nic_control-p1,routerDALL,control_net_DALL,100,auto,,12:54:D2:91:70:D6,eth1,eth1,10.133.132.3
routerDALL-nic_local-p1,routerDALL,net_localDALL,100,fablib,,12:CE:08:C6:E4:D0,eth3,eth3,192.168.1.1
routerDALL-DALL-WASH-link1-p1,routerDALL,DALL-WASH-link1,100,fablib,,16:76:D8:95:48:68,eth4,eth4,192.168.101.1
routerWASH-DALL-WASH-link1-p1,routerWASH,DALL-WASH-link1,100,fablib,,06:40:40:74:22:A4,eth2,eth2,192.168.101.2
routerWASH-nic_local-p1,routerWASH,net_localWASH,100,fablib,,06:2C:82:9C:93:8F,eth1,eth1,192.168.2.1
routerWASH-WASH-UTAH-link2-p1,routerWASH,WASH-UTAH-link2,100,fablib,,06:C9:B7:7A:B3:9E,eth4,eth4,192.168.102.1
routerWASH-nic_control-p1,routerWASH,control_net_WASH,100,auto,,06:55:70:3B:A3:4F,eth3,eth3,10.133.2.3
routerUTAH-WASH-UTAH-link2-p1,routerUTAH,WASH-UTAH-link2,100,fablib,,02:07:53:53:05:88,eth1,eth1,192.168.102.2
routerUTAH-nic_local-p1,routerUTAH,net_localUTAH,100,fablib,,12:45:1E:60:73:DB,eth3,eth3,192.168.3.1



Time to print interfaces 1039 seconds


'd55472a4-e574-4391-9ea1-0c9427ea8686'

## Config the Routers

In [15]:
for node in slice.get_nodes():
    if 'router' in node.get_name():

        print(node.get_name())
        attributes = node.upload_directory('docker_containers','.')
        #attributes = node.upload_directory('node_tools','.')
        
        frr_user_data = node.get_user_data()['frr']
        iface_local = slice.get_interface(frr_user_data['local_iface'])
        iface_link1 = slice.get_interface(frr_user_data['link_ifaces'][0])
        iface_link2 = slice.get_interface(frr_user_data['link_ifaces'][1])
        
        #stdout, stderr = node.execute(f'chmod +x node_tools/frr_config_docker.sh && ./node_tools/frr_config_docker.sh {iface_link1.get_device_name()} {iface_link1.get_ip_addr()} {iface_link2.get_device_name()} {iface_link2.get_ip_addr()} {iface_local.get_device_name()} {iface_local.get_ip_addr()} 192.168.0.0')
        
        devs = ' '.join([iface.get_device_name() for iface in node.get_interfaces()])
        
        print(f"./docker_containers/fabric_frrouting/start.sh {devs}")
        print(f"./docker_containers/fabric_frrouting/node_tools/frr_config_docker.sh {iface_link1.get_device_name()} {iface_link1.get_ip_addr()} {iface_link2.get_device_name()} {iface_link2.get_ip_addr()} {iface_local.get_device_name()} {iface_local.get_ip_addr()} 192.168.0.0")

        node.execute(f"./docker_containers/fabric_frrouting/node_tools/frr_config_docker.sh {iface_link1.get_device_name()} {iface_link1.get_ip_addr()} {iface_link2.get_device_name()} {iface_link2.get_ip_addr()} {iface_local.get_device_name()} {iface_local.get_ip_addr()} 192.168.0.0 ; "
                     f"./docker_containers/fabric_frrouting/start.sh {devs} ; "
                      , quiet=True, output_file=f"{node.get_name()}.log")
        
        

        

routerDALL
./docker_containers/fabric_frrouting/start.sh eth2 eth1 eth3 eth4
./docker_containers/fabric_frrouting/node_tools/frr_config_docker.sh eth4 192.168.101.1 eth2 192.168.103.2 eth3 192.168.1.1 192.168.0.0
routerWASH
./docker_containers/fabric_frrouting/start.sh eth2 eth1 eth4 eth3
./docker_containers/fabric_frrouting/node_tools/frr_config_docker.sh eth2 192.168.101.2 eth4 192.168.102.1 eth1 192.168.2.1 192.168.0.0
routerUTAH
./docker_containers/fabric_frrouting/start.sh eth1 eth3 eth4 eth2
./docker_containers/fabric_frrouting/node_tools/frr_config_docker.sh eth1 192.168.102.2 eth2 192.168.103.1 eth3 192.168.3.1 192.168.0.0


## Config the Nodes

In [16]:
for node in slice.get_nodes():
    if 'router' not in node.get_name():
        print(node.get_name())
        attributes = node.upload_directory('docker_containers','.')
        #attributes = node.upload_directory('node_tools','.')
        
        devs = ' '.join([iface.get_device_name() for iface in node.get_interfaces()])
        node.execute(f"cd ~/docker_containers/fabric_multitool/; docker compose up -d ; "
                     f"docker exec fabric sudo ./tools/host_tune_redhat.sh ; "
                     #f"./start.sh {devs} ; "
                      , quiet=True, output_file=f"{node.get_name()}.log")
        

dall1
wash1
utah1


## Run the Experiment

We will just test `ping` RTT and look at `tracepath`. Your experiment should be more interesting!

Notice that if you run this quickly and repeatedly run this test against a specific target, you may see changes to the tracepath.  Initially the ping may even fail.  Why do you think this is happening?


In [ ]:
try:
    source_node_name =  f'{node_base_name}_site1_1'
   
    source_node = slice.get_node(name=source_node_name)
    for node_name,target_ip in local_dataplane_ips.items():
        print(f"Testing target node: {node_name}, target IP: {target_ip}")
    
        stdout, stderr = node.execute(f'ping -c 5 {target_ip}')

        stdout, stderr = node.execute(f'tracepath {target_ip}')
    
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")